# ANÁLISIS PROFUNDO: Eventos E0 (2004-2025) - FIXED

**Objetivo**: Análisis exhaustivo de TODOS los eventos en watchlists (no solo info_rich)

**CORRECCIÓN CRÍTICA**: Analizamos TODOS los registros para ver combinaciones reales de filtros

**Preguntas clave**:
1. ¿Qué combinaciones de filtros existen en TODOS los datos? (4, 3, 2, 1, 0)
2. ¿Cuántos eventos E0 (4 filtros) tiene cada ticker?
3. ¿Cuándo se cumplen los 4 filtros a la vez? (fecha exacta para TradingView)
4. ¿Cuál es el mejor día del mes?
5. ¿Cuál es el mejor mes del año?
6. ¿Cuál fue el mejor año?

**Filtros E0 evaluados**:
- **F1**: RVOL ≥ 2.0 (volumen relativo)
- **F2**: |%chg| ≥ 15% (cambio precio)
- **F3**: $vol ≥ $5M (dollar volume)
- **F4**: Precio $0.20-$20 (rango small caps)

---

In [ ]:
import polars as pl
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Config
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 11

# Paths
PROJECT_ROOT = Path(r"D:\04_TRADING_SMALLCAPS")
WATCHLISTS = PROJECT_ROOT / "processed" / "universe" / "info_rich" / "daily"
OUTPUT_DIR = PROJECT_ROOT / "01_DayBook" / "fase_01" / "C_v2_ingesta_tiks_2004_2025" / "notebooks"

print("✅ Setup complete")
print(f"📂 Watchlists: {WATCHLISTS}")
print(f"📂 Output: {OUTPUT_DIR}")

---

## 1. CARGA DE DATOS: TODOS los Registros (NO solo info_rich)

In [ ]:
print("="*80)
print("CARGANDO TODOS LOS REGISTROS DE WATCHLISTS")
print("="*80)

# Cargar TODOS los registros (no filtrar por info_rich)
print("\n📊 Cargando TODOS los registros...")
df_all = pl.scan_parquet(WATCHLISTS / "date=*" / "watchlist.parquet").collect()

print(f"\n✅ Total registros cargados: {len(df_all):,}")
print(f"   Columnas: {df_all.columns}")
print(f"   Memoria: {df_all.estimated_size('mb'):.2f} MB")
print(f"   Tickers únicos: {df_all['ticker'].n_unique():,}")
print(f"   Días únicos: {df_all['trading_day'].n_unique():,}")
print(f"   Rango temporal: {df_all['trading_day'].min()} → {df_all['trading_day'].max()}")

# Verificar columna info_rich
e0_count = (df_all['info_rich'] == True).sum()
print(f"\n📊 Eventos marcados como E0 (info_rich=True): {e0_count:,} ({e0_count/len(df_all)*100:.2f}%)")

# Sample
print(f"\n📋 Sample (5 filas random):")
print(df_all.sample(5).select(['ticker', 'trading_day', 'rvol30', 'pctchg_d', 'dollar_vol_d', 'close_d', 'info_rich']))

---

## 2. EVALUAR FILTROS EN TODOS LOS REGISTROS

**CLAVE**: Evaluamos filtros en TODOS los datos, no solo en info_rich=True

In [ ]:
print("="*80)
print("EVALUACIÓN DE FILTROS EN TODOS LOS REGISTROS")
print("="*80)

# Evaluar cada filtro en TODOS los registros
df_all = df_all.with_columns([
    (pl.col('rvol30') >= 2.0).fill_null(False).alias('f1_rvol'),
    (pl.col('pctchg_d').abs() >= 0.15).fill_null(False).alias('f2_pctchg'),
    (pl.col('dollar_vol_d') >= 5_000_000).fill_null(False).alias('f3_dvol'),
    ((pl.col('close_d') >= 0.20) & (pl.col('close_d') <= 20.0)).fill_null(False).alias('f4_price')
])

# Contar cuántos filtros cumple cada registro
df_all = df_all.with_columns(
    (pl.col('f1_rvol').cast(pl.Int32) + 
     pl.col('f2_pctchg').cast(pl.Int32) + 
     pl.col('f3_dvol').cast(pl.Int32) + 
     pl.col('f4_price').cast(pl.Int32)).alias('num_filtros')
)

# Estadísticas de filtros individuales
total = len(df_all)
f1_count = df_all['f1_rvol'].sum()
f2_count = df_all['f2_pctchg'].sum()
f3_count = df_all['f3_dvol'].sum()
f4_count = df_all['f4_price'].sum()

print(f"\nTotal registros: {total:,}")
print(f"\n{'Filtro':<30} {'Cumple':<15} {'%':<10}")
print("-"*55)
print(f"{'F1: RVOL ≥ 2.0':<30} {f1_count:>14,} {f1_count/total*100:>9.2f}%")
print(f"{'F2: |%chg| ≥ 15%':<30} {f2_count:>14,} {f2_count/total*100:>9.2f}%")
print(f"{'F3: $vol ≥ $5M':<30} {f3_count:>14,} {f3_count/total*100:>9.2f}%")
print(f"{'F4: Precio $0.20-$20':<30} {f4_count:>14,} {f4_count/total*100:>9.2f}%")

# Gráfico
fig, ax = plt.subplots(figsize=(12, 6))
filtros = ['F1\nRVOL≥2.0', 'F2\n|%chg|≥15%', 'F3\n$vol≥$5M', 'F4\nPrecio\n$0.20-$20']
counts = [f1_count, f2_count, f3_count, f4_count]
percentages = [c/total*100 for c in counts]

bars = ax.bar(filtros, percentages, color=['steelblue', 'coral', 'green', 'purple'], alpha=0.7)
ax.set_ylabel('% de TODOS los Registros que Cumplen', fontsize=12)
ax.set_title('Cumplimiento de Filtros Individuales (TODOS los registros)', fontsize=14, fontweight='bold')
ax.set_ylim(0, max(percentages) * 1.15)
ax.grid(axis='y', alpha=0.3)

for bar, pct, cnt in zip(bars, percentages, counts):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.5,
            f'{pct:.2f}%\n({cnt:,})',
            ha='center', va='bottom', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'e0_filtros_individuales_FIXED.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Gráfico guardado: e0_filtros_individuales_FIXED.png")

---

## 3. COMBINACIONES DE FILTROS

**Pregunta**: De TODOS los registros, ¿cuántos cumplen 4, 3, 2, 1, 0 filtros?

In [ ]:
print("="*80)
print("COMBINACIONES DE FILTROS (TODOS LOS REGISTROS)")
print("="*80)

# Distribución por número de filtros
dist_filtros = df_all.group_by('num_filtros').agg(pl.count().alias('count')).sort('num_filtros', descending=True)

print(f"\n{'# Filtros':<15} {'Eventos':<15} {'%':<10}")
print("-"*40)
for row in dist_filtros.iter_rows(named=True):
    num = row['num_filtros']
    cnt = row['count']
    pct = cnt/total*100
    label = "(E0 = info_rich)" if num == 4 else ""
    print(f"{num:<15} {cnt:>14,} {pct:>9.2f}% {label}")

# Verificar que 4 filtros == info_rich
eventos_4_filtros = (df_all['num_filtros'] == 4).sum()
eventos_info_rich = (df_all['info_rich'] == True).sum()
print(f"\n🔍 VERIFICACIÓN:")
print(f"   Eventos con 4 filtros: {eventos_4_filtros:,}")
print(f"   Eventos info_rich=True: {eventos_info_rich:,}")
print(f"   Match: {'✅' if eventos_4_filtros == eventos_info_rich else '❌'}")

# Gráfico
fig, ax = plt.subplots(figsize=(12, 7))
dist_pd = dist_filtros.to_pandas().sort_values('num_filtros')

colors = ['gray', 'red', 'orange', 'yellow', 'lightgreen', 'darkgreen']
bars = ax.bar(dist_pd['num_filtros'].astype(str), dist_pd['count'], 
              color=colors[:len(dist_pd)], alpha=0.7, edgecolor='black', linewidth=1.5)

ax.set_xlabel('Número de Filtros Cumplidos', fontsize=12)
ax.set_ylabel('Cantidad de Registros', fontsize=12)
ax.set_title('Distribución de TODOS los Registros por Número de Filtros', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
ax.set_yscale('log')  # Escala log para ver mejor

for bar in bars:
    height = bar.get_height()
    label = bar.get_x() + bar.get_width()/2.
    if label == 4:  # Destacar eventos E0
        ax.text(bar.get_x() + bar.get_width()/2., height * 1.2,
                f'{int(height):,}\n({height/total*100:.2f}%)\n⭐ E0',
                ha='center', va='bottom', fontweight='bold', fontsize=10, color='darkgreen')
    else:
        ax.text(bar.get_x() + bar.get_width()/2., height * 1.2,
                f'{int(height):,}\n({height/total*100:.2f}%)',
                ha='center', va='bottom', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'e0_distribucion_filtros_FIXED.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Gráfico guardado: e0_distribucion_filtros_FIXED.png")

---

## 4. ANÁLISIS DETALLADO: Combinaciones Específicas

**Pregunta**: ¿Qué combinaciones específicas de 3, 2, 1 filtros son más comunes?

In [ ]:
print("="*80)
print("COMBINACIONES ESPECÍFICAS DE FILTROS")
print("="*80)

# Crear columna de combinación como string
df_all = df_all.with_columns(
    (pl.when(pl.col('f1_rvol')).then(pl.lit('F1')).otherwise(pl.lit('')) +
     pl.when(pl.col('f2_pctchg')).then(pl.lit('+F2')).otherwise(pl.lit('')) +
     pl.when(pl.col('f3_dvol')).then(pl.lit('+F3')).otherwise(pl.lit('')) +
     pl.when(pl.col('f4_price')).then(pl.lit('+F4')).otherwise(pl.lit(''))
    ).str.replace_all('^\\+', '').str.replace_all('\\+$', '').alias('combinacion')
)

# Eventos con 3 filtros
print("\n📊 EVENTOS CON 3 FILTROS (falta 1):")
df_3 = df_all.filter(pl.col('num_filtros') == 3)
comb_3 = df_3.group_by('combinacion').agg(pl.count().alias('count')).sort('count', descending=True)
print(comb_3.head(10))

# Eventos con 2 filtros
print("\n📊 EVENTOS CON 2 FILTROS:")
df_2 = df_all.filter(pl.col('num_filtros') == 2)
comb_2 = df_2.group_by('combinacion').agg(pl.count().alias('count')).sort('count', descending=True)
print(comb_2.head(10))

# Eventos con 1 filtro
print("\n📊 EVENTOS CON 1 FILTRO:")
df_1 = df_all.filter(pl.col('num_filtros') == 1)
comb_1 = df_1.group_by('combinacion').agg(pl.count().alias('count')).sort('count', descending=True)
print(comb_1)

print("\n💡 INTERPRETACIÓN:")
print("   - Combinaciones más comunes muestran qué filtros se cumplen juntos")
print("   - Útil para ajustar thresholds y crear filtros alternativos")

---

## 5. EVENTOS E0 (4 FILTROS): Análisis por Ticker

In [ ]:
print("="*80)
print("EVENTOS E0 (4 FILTROS) POR TICKER")
print("="*80)

# Filtrar SOLO eventos con 4 filtros (E0)
df_e0 = df_all.filter(pl.col('num_filtros') == 4)

print(f"\n✅ Total eventos E0 (4 filtros): {len(df_e0):,}")
print(f"   Tickers únicos: {df_e0['ticker'].n_unique():,}")

# Estadísticas por ticker
by_ticker = (df_e0.group_by('ticker')
             .agg(pl.count().alias('total_eventos_e0'))
             .sort('total_eventos_e0', descending=True))

print(f"\n📊 Estadísticas de eventos E0 por ticker:")
eventos_stats = by_ticker['total_eventos_e0']
print(f"   Min: {eventos_stats.min()}")
print(f"   Q1: {eventos_stats.quantile(0.25):.0f}")
print(f"   Mediana: {eventos_stats.median():.0f}")
print(f"   Q3: {eventos_stats.quantile(0.75):.0f}")
print(f"   Max: {eventos_stats.max()}")
print(f"   Media: {eventos_stats.mean():.2f}")

# Distribución
print(f"\n📊 Distribución de tickers por # eventos E0:")
bins = [1, 2, 5, 10, 20, 50, 100]
for i in range(len(bins)-1):
    count = ((eventos_stats >= bins[i]) & (eventos_stats < bins[i+1])).sum()
    print(f"   {bins[i]}-{bins[i+1]-1} eventos: {count:,} tickers ({count/len(by_ticker)*100:.1f}%)")
count_100plus = (eventos_stats >= 100).sum()
print(f"   100+ eventos: {count_100plus:,} tickers ({count_100plus/len(by_ticker)*100:.1f}%)")

# TOP 30
top30 = by_ticker.head(30)
print(f"\n📊 TOP 30 Tickers con Más Eventos E0:")
print(top30)

# Gráfico
fig, ax = plt.subplots(figsize=(14, 10))
top30_pd = top30.to_pandas()

ax.barh(range(len(top30_pd)), top30_pd['total_eventos_e0'], color='teal', alpha=0.7)
ax.set_yticks(range(len(top30_pd)))
ax.set_yticklabels(top30_pd['ticker'])
ax.set_xlabel('Número de Eventos E0', fontsize=12)
ax.set_title('TOP 30 Tickers con Más Eventos E0 (4 filtros)', fontsize=14, fontweight='bold')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

for i, count in enumerate(top30_pd['total_eventos_e0']):
    ax.text(count + 0.5, i, f'{count}', va='center', fontweight='bold')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'e0_top30_tickers_FIXED.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Gráfico guardado: e0_top30_tickers_FIXED.png")

---

## 6. ANÁLISIS TEMPORAL: ¿Cuándo ocurren los eventos E0?

In [ ]:
print("="*80)
print("ANÁLISIS TEMPORAL: EVENTOS E0")
print("="*80)

# Extraer componentes temporales
df_e0 = df_e0.with_columns([
    pl.col('trading_day').str.slice(0, 4).alias('year'),
    pl.col('trading_day').str.slice(5, 2).alias('month'),
    pl.col('trading_day').str.slice(8, 2).alias('day')
])

total_e0 = len(df_e0)

# Por año
by_year = df_e0.group_by('year').agg(pl.count().alias('eventos')).sort('year')
print(f"\n📊 DISTRIBUCIÓN POR AÑO:")
for row in by_year.iter_rows(named=True):
    print(f"   {row['year']}: {row['eventos']:>6,} eventos ({row['eventos']/total_e0*100:>5.2f}%)")

# Por mes
by_month = df_e0.group_by('month').agg(pl.count().alias('eventos')).sort('month')
month_names = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
print(f"\n📊 DISTRIBUCIÓN POR MES:")
for row in by_month.iter_rows(named=True):
    month_name = month_names[int(row['month'])-1]
    print(f"   {month_name}: {row['eventos']:>6,} eventos ({row['eventos']/total_e0*100:>5.2f}%)")

# Por día del mes
by_day = df_e0.group_by('day').agg(pl.count().alias('eventos')).sort('day')
print(f"\n📊 TOP 10 DÍAS DEL MES:")
for row in by_day.sort('eventos', descending=True).head(10).iter_rows(named=True):
    print(f"   Día {row['day']}: {row['eventos']:>6,} eventos ({row['eventos']/total_e0*100:>5.2f}%)")

# Gráficos
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(16, 14))

# Año
by_year_pd = by_year.to_pandas()
ax1.bar(by_year_pd['year'], by_year_pd['eventos'], color='steelblue', alpha=0.7)
ax1.set_ylabel('Eventos E0', fontsize=12)
ax1.set_title('Eventos E0 por Año', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Mes
by_month_pd = by_month.to_pandas()
by_month_pd['month_name'] = by_month_pd['month'].apply(lambda x: month_names[int(x)-1])
ax2.bar(by_month_pd['month_name'], by_month_pd['eventos'], color='coral', alpha=0.7)
ax2.set_ylabel('Eventos E0', fontsize=12)
ax2.set_title('Eventos E0 por Mes del Año', fontsize=14, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

# Día
by_day_pd = by_day.to_pandas()
ax3.bar(by_day_pd['day'], by_day_pd['eventos'], color='purple', alpha=0.7)
ax3.set_xlabel('Día del Mes', fontsize=12)
ax3.set_ylabel('Eventos E0', fontsize=12)
ax3.set_title('Eventos E0 por Día del Mes', fontsize=14, fontweight='bold')
ax3.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'e0_distribucion_temporal_FIXED.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Gráfico guardado: e0_distribucion_temporal_FIXED.png")

# Mejores períodos
best_year = by_year_pd.loc[by_year_pd['eventos'].idxmax()]
best_month = by_month_pd.loc[by_month_pd['eventos'].idxmax()]
best_day = by_day_pd.loc[by_day_pd['eventos'].idxmax()]

print(f"\n🏆 MEJORES PERÍODOS:")
print(f"   Mejor año: {best_year['year']} ({best_year['eventos']:,} eventos)")
print(f"   Mejor mes: {best_month['month_name']} ({best_month['eventos']:,} eventos)")
print(f"   Mejor día del mes: Día {best_day['day']} ({best_day['eventos']:,} eventos)")

---

## 7. EXPORTAR CSVs PARA TRADINGVIEW

In [ ]:
print("="*80)
print("EXPORTANDO CSVs")
print("="*80)

# 1. Eventos E0 (4 filtros) para TradingView
df_export_e0 = df_e0.select([
    'ticker',
    'trading_day',
    'rvol30',
    'pctchg_d',
    'dollar_vol_d',
    'close_d',
    'vol_d',
    'vwap_d'
]).sort(['trading_day', 'ticker'])

csv_e0 = OUTPUT_DIR / 'eventos_E0_4_filtros_TRADINGVIEW.csv'
df_export_e0.write_csv(csv_e0)
print(f"\n✅ CSV 1: {csv_e0.name}")
print(f"   Total eventos E0: {len(df_export_e0):,}")

# 2. Estadísticas por ticker
by_ticker_detailed = (df_e0.group_by('ticker')
                      .agg([
                          pl.count().alias('total_eventos_e0'),
                          pl.col('rvol30').mean().alias('rvol30_promedio'),
                          pl.col('pctchg_d').abs().mean().alias('pctchg_promedio'),
                          pl.col('dollar_vol_d').mean().alias('dvol_promedio'),
                          pl.col('close_d').mean().alias('precio_promedio')
                      ])
                      .sort('total_eventos_e0', descending=True))

csv_ticker = OUTPUT_DIR / 'estadisticas_por_ticker_E0.csv'
by_ticker_detailed.write_csv(csv_ticker)
print(f"\n✅ CSV 2: {csv_ticker.name}")
print(f"   Total tickers: {len(by_ticker_detailed):,}")

# 3. Combinaciones de filtros
dist_filtros_export = df_all.group_by(['num_filtros', 'combinacion']).agg(
    pl.count().alias('count')
).sort(['num_filtros', 'count'], descending=[True, True])

csv_comb = OUTPUT_DIR / 'combinaciones_filtros_E0.csv'
dist_filtros_export.write_csv(csv_comb)
print(f"\n✅ CSV 3: {csv_comb.name}")
print(f"   Total combinaciones únicas: {len(dist_filtros_export):,}")

print(f"\n💡 USO EN TRADINGVIEW:")
print(f"   1. Abrir TradingView")
print(f"   2. Buscar ticker + fecha de eventos_E0_4_filtros_TRADINGVIEW.csv")
print(f"   3. Verificar visualmente que se cumplen los 4 filtros")

---

## 8. RESUMEN EJECUTIVO

In [ ]:
print("\n" + "="*80)
print("RESUMEN EJECUTIVO - ANÁLISIS EVENTOS E0 (FIXED)")
print("="*80)

print(f"\n📊 DATOS GENERALES:")
print(f"   Total registros analizados: {total:,}")
print(f"   Rango temporal: {df_all['trading_day'].min()} → {df_all['trading_day'].max()}")

print(f"\n📊 CUMPLIMIENTO DE FILTROS (de todos los registros):")
print(f"   F1 (RVOL≥2.0): {f1_count:,} ({f1_count/total*100:.2f}%)")
print(f"   F2 (|%chg|≥15%): {f2_count:,} ({f2_count/total*100:.2f}%)")
print(f"   F3 ($vol≥$5M): {f3_count:,} ({f3_count/total*100:.2f}%)")
print(f"   F4 (Precio $0.20-$20): {f4_count:,} ({f4_count/total*100:.2f}%)")

print(f"\n📊 DISTRIBUCIÓN POR # FILTROS:")
for row in dist_filtros.iter_rows(named=True):
    num = row['num_filtros']
    cnt = row['count']
    label = "⭐ E0 (info_rich)" if num == 4 else ""
    print(f"   {num} filtros: {cnt:>10,} ({cnt/total*100:>6.2f}%) {label}")

print(f"\n📊 EVENTOS E0 (4 FILTROS):")
print(f"   Total: {total_e0:,}")
print(f"   Tickers únicos: {df_e0['ticker'].n_unique():,}")
print(f"   % del universo: {total_e0/total*100:.2f}%")

print(f"\n🏆 MEJORES PERÍODOS:")
print(f"   Mejor año: {best_year['year']} ({best_year['eventos']:,} eventos)")
print(f"   Mejor mes: {best_month['month_name']} ({best_month['eventos']:,} eventos)")
print(f"   Mejor día del mes: Día {best_day['day']} ({best_day['eventos']:,} eventos)")

print(f"\n📁 ARCHIVOS GENERADOS:")
print(f"   ✅ eventos_E0_4_filtros_TRADINGVIEW.csv ({len(df_export_e0):,} eventos)")
print(f"   ✅ estadisticas_por_ticker_E0.csv ({len(by_ticker_detailed):,} tickers)")
print(f"   ✅ combinaciones_filtros_E0.csv ({len(dist_filtros_export):,} combinaciones)")
print(f"   ✅ e0_filtros_individuales_FIXED.png")
print(f"   ✅ e0_distribucion_filtros_FIXED.png")
print(f"   ✅ e0_top30_tickers_FIXED.png")
print(f"   ✅ e0_distribucion_temporal_FIXED.png")

print(f"\n💡 INSIGHT CLAVE:")
print(f"   Los eventos E0 (4 filtros) representan solo {total_e0/total*100:.2f}% del universo")
print(f"   Esto valida el enfoque event-driven: focalizamos en días info-rich")

print("\n" + "="*80)